In [1]:
%pip install ultralytics > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ultralytics
import torch
import os
import json
import yaml
import pandas as pd
import shutil

In [3]:
# Training parameters

num_epochs = 100 # number of training epochs
model_name = "yolov8s.pt"
sep_val = False # whether or not to separate images for valuation
val_step = 6 # if separating valuation, then every i-th training image is chosen

dataset_path = "/kaggle/input/kalev-candy-detection-competition/"

In [4]:
# Create necessary folder structure for yolo model

os.mkdir('/kaggle/working/dataset')
os.mkdir('/kaggle/working/dataset/images')
os.mkdir('/kaggle/working/dataset/labels')
os.mkdir('/kaggle/working/dataset/images/train')
os.mkdir('/kaggle/working/dataset/images/val')
os.mkdir('/kaggle/working/dataset/labels/train')
os.mkdir('/kaggle/working/dataset/labels/val')

In [5]:
# Create yolo-style annotations files

fromDir = '/kaggle/input/kalev-candy-detection-competition/train/'

with open(dataset_path + 'train_labels.json') as f:
    obj = json.load(f)

class_labels = {'kaseke': 0, 'maiuspala': 1, 'pralinee': 2, 'ananass': 3, 'mesikapp': 4, 'tiina': 5}

# iterate through the labels from the json object: for each create a respective .txt file in the yolo annotation format

for idx, case in enumerate(obj):
    
    if sep_val and idx % val_step == 0:
        destDir = 'val'
    else:
        destDir = 'train'
        
    toDirectory = f'/kaggle/working/dataset/images/{destDir}/{case["image"]}'
    
    shutil.copyfile(fromDir + case["image"], toDirectory)

    with open(f'/kaggle/working/dataset/labels/{destDir}/{case["image"].split(".")[0]}.txt', 'w') as f:

        num_boxes = len(case['bboxes'])
        for box in case['bboxes']:
            x = box['x']
            y = box['y']
            width = box['width']
            height = box['height']
            x += width / 2
            y += height / 2

            f.write(f'{class_labels[box["label"]]} {x} {y} {width} {height}')
            if num_boxes > 1:
                f.write('\n')
                num_boxes -= 1

In [6]:
# Create config.yaml file with the necessary paths and classes

if sep_val:
    val_dir = 'val'
else:
    val_dir = 'train'

data = {
    'path': '/kaggle/working/dataset',
    'train': 'images/train',
    'val': f'images/{val_dir}', # if not separating valuation images then just use training images
    "names": {
          0: 'kaseke',
          1: 'maiuspala',
          2: 'pralinee',
          3: 'ananass',
          4: 'mesikapp',
          5: 'tiina'}
}

with open('config.yaml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

In [7]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")

GPU is available


In [8]:
model = ultralytics.YOLO(model_name)

model.to(device)

100%|██████████| 21.5M/21.5M [00:00<00:00, 144MB/s] 


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [9]:
# Currently not using wandb
os.environ['WANDB_DISABLED'] = 'true'

In [10]:
model.train(data="config.yaml", epochs=num_epochs, device='0')

Ultralytics YOLOv8.1.47 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=config.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]
2024-04-11 21:10:49,858	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-11 21:10:50,689	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-11 21:10:53.062662: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 21:10:53.062773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 21:10:53.194065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS facto

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

wandb: Tracking run with wandb version 0.16.5
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...


100 epochs completed in 1.237 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:22<00:00,  1.73s/it]


                   all        401       1192      0.999          1      0.995      0.978
                kaseke        401        204          1      0.998      0.995      0.974
             maiuspala        401        206          1          1      0.995      0.976
              pralinee        401        132      0.999          1      0.995      0.969
               ananass        401        188      0.999          1      0.995      0.984
              mesikapp        401        207      0.999          1      0.995      0.984
                 tiina        401        255      0.999          1      0.995      0.983
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg1 ▃▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                  lr/pg2 ▃▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:        metrics/mAP50(B) ▁▆▇▇████████████████████████████████████
wandb:     metrics/mAP50-95(B) ▁▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
wandb:    metrics/precision(B) ▁▅▆▇▇▇▇█████████████████████████████████
wandb:       metrics/recall(B) ▁▄▆▆▇▇▇█████████████████████████████████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▇▆▅▆▅▅▅▅▅▄▄▄▄▄▃▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▁▁▁▁
wandb:          train/cls_loss █▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
wandb:          train/dfl_loss █▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▁▁▁▁
wandb:            val/box_loss ▆█▇▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7997fbe42440>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [11]:
# paths to all the test images
test_images = [dataset_path + f'test/test_img_{idx}.jpg' for idx in range(100)]

In [12]:
# make the predictions
results = model(test_images)


0: 640x640 1 pralinee, 1 ananass, 1 mesikapp, 4.3ms
1: 640x640 1 kaseke, 1 maiuspala, 1 pralinee, 1 ananass, 1 mesikapp, 1 tiina, 4.3ms
2: 640x640 1 maiuspala, 1 mesikapp, 4.3ms
3: 640x640 1 kaseke, 1 pralinee, 4.3ms
4: 640x640 1 kaseke, 4.3ms
5: 640x640 1 maiuspala, 4.3ms
6: 640x640 1 maiuspala, 1 ananass, 4.3ms
7: 640x640 1 kaseke, 1 maiuspala, 1 pralinee, 1 ananass, 1 mesikapp, 1 tiina, 4.3ms
8: 640x640 1 kaseke, 1 maiuspala, 4.3ms
9: 640x640 1 mesikapp, 4.3ms
10: 640x640 (no detections), 4.3ms
11: 640x640 1 maiuspala, 4.3ms
12: 640x640 1 maiuspala, 1 pralinee, 4.3ms
13: 640x640 1 maiuspala, 1 pralinee, 1 tiina, 4.3ms
14: 640x640 2 kasekes, 1 maiuspala, 1 ananass, 1 mesikapp, 4.3ms
15: 640x640 1 ananass, 4.3ms
16: 640x640 2 kasekes, 3 mesikapps, 4.3ms
17: 640x640 2 kasekes, 3 maiuspalas, 1 ananass, 3 mesikapps, 2 tiinas, 4.3ms
18: 640x640 2 kasekes, 3 maiuspalas, 2 pralinees, 1 ananass, 3 mesikapps, 2 tiinas, 4.3ms
19: 640x640 2 kasekes, 3 maiuspalas, 3 pralinees, 1 ananass, 4 mesi

In [13]:
# Create the empty solution dataframe
solutions = pd.DataFrame({'Id': range(100), 'bboxes': [[]] * 100})

In [14]:
# Parse the predictions into the dataframe

idx = 0
for result in results:

    boxes = result.boxes.xyxy.tolist()
    classes = result.boxes.cls.tolist()
    names = result.names
    confidences = result.boxes.conf.tolist()

    bounding_boxes = []

    # Iterate through the results
    for box, cls, conf in zip(boxes, classes, confidences):
        x1, y1, x2, y2 = box
        confidence = conf
        detected_class = cls
        name = names[int(cls)]
        bounding_boxes.append({'x': x1, 'y': y1, 'width': x2 - x1, 'height': y2 - y1, 'label': name})

    solutions.at[idx, 'bboxes'] = bounding_boxes
    idx += 1

In [15]:
solutions.to_csv('submission.csv', index=False)